# 1. 经典的单例模式

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

if __name__ == '__main__':
    s = Singleton()
    print("Object created", s)
    s1 = Singleton()
    print("Object created", s1)

Object created <__main__.Singleton object at 0x7f1e980cf668>
Object created <__main__.Singleton object at 0x7f1e980cf668>


1. 只允许Singleton类生成一个实例
2. 如果已经有一个实例了，则返回同一个对象

## 其中涉及到不懂的内容有
1. ```__new__``` 是Python用于实例化对象的特殊方法，用来控制对象的创建。
2. hassattr 用于检查cls对象是否拥有instance属性，该属性的作用是用来检查该类是否已经生成一个对象。
3. cls 代表该类本身，他可以用来调用自己属性和类自己的方法。
4. super 用来调用父类的方法。

# 2. 单例模式中的懒汉式实例化

In [11]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print('__init__ method called.')
        else:
            print('Instance already created: ', self.getInstance())
        
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton
        return cls.__instance

if __name__ == '__main__':
    s = Singleton()  # class initialized, but object not created
    print("Object created", Singleton.getInstance())  # Object gets created
    s1 = Singleton()  # instance already created

__init__ method called.
Object created <class '__main__.Singleton'>
Instance already created:  <class '__main__.Singleton'>


1. 懒汉式实例化能够确保在实际需要时才创建对象
2. 节约资源
## 其中涉及到不懂的内容：
1. ```@classmethod``` 修饰符对应的函数不需要实例化，不需要self参数，但第一个参数必须是表示自身类的cls参数，可以掉类的属性，方法和实例化对象

# 3. 模块级别的单例模式

1. 检查一个Python模块是否已经导入。
2. 如果已经导入，则返回该模块的对象， 如果还没有导入，则导入该模块，并实例化。
3. 当模块被导入的时候，它就会被初始化。然而，当同一个模块被再次导入的时候，它不会再次初始化。

# 4. Monostate单例(态)模式

In [15]:
class Borg:
    __shared_state = {'1':'2'}
    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass

if __name__ == '__main__':
    b = Borg()
    b1 = Borg()
    b.x = 4
    print("Borg Object 'b': ", b)
    print("Borg Object 'b1': ", b1)
    print("Borg State 'b': ", b.__dict__)
    print("Borg State 'b1': ", b1.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x7f1e980d5978>
Borg Object 'b1':  <__main__.Borg object at 0x7f1e980d5ac8>
Borg State 'b':  {'1': '2', 'x': 4}
Borg State 'b1':  {'1': '2', 'x': 4}


1. 所有对象共享相同状态被成为Monostate(单态)模式。

## 其中涉及到不懂的内容：
1. ```__dict__``` 它是python的一个特殊变量用来存储一个类所有对象的状态
2. ```__dict__``` 用于存储类的静态函数，类函数，普通函数，全局变量以及一些内置的属性。

除此之外，也可以通过重写__new__方法来实现Borg模式。

In [16]:
class Borg(object):
    _shared_state = {}
    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

if __name__ == '__main__':
    b = Borg()
    b1 = Borg()
    b.x = 4
    print("Borg Object 'b': ", b)
    print("Borg Object 'b1': ", b1)
    print("Borg State 'b': ", b.__dict__)
    print("Borg State 'b1': ", b1.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x7f1e980d5e10>
Borg Object 'b1':  <__main__.Borg object at 0x7f1e980d5978>
Borg State 'b':  {'x': 4}
Borg State 'b1':  {'x': 4}


# 5. 单例和元类

In [19]:
class MyInit(type):
    def __call__(cls, *args, **kwds):
        print("***** Here`s My int *****")
        print("Now do whatever you want with those objects....")
        return type.__call__(cls, *args, **kwds)

class int(metaclass=MyInit):
    def __init__(self, x, y):
        self.x = x
        self.y = y

if __name__ == '__main__':
    i = int(4, 5)
    print(i.__dict__)
    print("---- Inside Value ----")
    print(i.x)
    print(i.y)

***** Here`s My int *****
Now do whatever you want with those objects....
{'x': 4, 'y': 5}
---- Inside Value ----
4
5


## 其中涉及到不懂的内容：
* Python通过A=type(name, bases, dict)来创建元类. 其中name是类的名称，base是基类，dict是属性变量。
* 其中MyInit为元类，重新定义int的元类需要对metaclass进行赋值，默认为type
* ```__new__(cls, *args, **kwargs)```创建对象时调用，且第一个参数为cls即```class```该类本身
* ``` __init__(self, *args, **kargs)```创建完成对象后调用，对于当前对象的实例进行一些初始化，无返回值，第一个参数为self，即对象本身。
* ``` __call(self, *args, **kargs)``` 把这个类型的对象当函数来使用，相当于重载了```()```运算符。

由于元类在对象创建和对象实例化有更多的控制权，所以他可以用于创建单例
* 覆盖__new__方法
* 覆盖__init__方法
* 覆盖__call__方法

In [21]:
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
        
class Logger(metaclass=MetaSingleton):
    pass

if __name__ == "__main__":
    logger1 = Logger()
    logger2 = Logger()
    print(logger1, logger2)

<__main__.Logger object at 0x7f1e98065390> <__main__.Logger object at 0x7f1e98065390>


# 6.单例模式 I

In [25]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj

if __name__ == "__main__":
    db1 = Database().connect()
    db2 = Database().connect()
    print("Database Object DB1", db1)
    print("Database Object DB2", db2)

Database Object DB1 <sqlite3.Cursor object at 0x7f1e980c2420>
Database Object DB2 <sqlite3.Cursor object at 0x7f1e980c2420>


# 7.单例模式 II

In [27]:
class HealthCheck:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, cls).__new__(cls, *args, **kwargs)
        return HealthCheck._instance
    
    def __init__(self):
        self._servers = []
    
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
        
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")
        
if __name__ == "__main__":
    hc1 = HealthCheck()
    hc2 = HealthCheck()
    hc1.addServer()
    print("Schedule health check for servers (1)..")
    for i in range(4):
        print("Checking ", hc1._servers[i])
    hc2.changeServer()
    print("Schedule health check for servers (2)..")
    for i in range(4):
        print("Checking ", hc2._servers[i])

Schedule health check for servers (1)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 4
Schedule health check for servers (2)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 5


# 单例模式总结

## 存在一下缺点
* 全局变量可能在某处已经被误改，但是开发人员仍然认为他们没有发生变化，而该变量还在应用程序的其他位置被使用。
* 可能会对同一对象创建多个引用。由于单例只创建一个对象，因此这种情况下会对同一个对象创建多个引用。
* 所有依赖全部变量的类都会由于一个类的改变而紧密耦合为全局数据，从而可能在无意中影响另一个类。

## 知识点
* 在许多实际用用程序中，我们只需要创建一个对象，如线程池，缓存，对话框，注册表设置等。如果我们为每个应用程序创建多实例，则会导致资源的过度使用。单例模式在这种情况下工作的很好。
* 单例是一种经过时间考研的成熟方法，能够在不带来太多缺陷的情况下提供全局访问点。
* 当然，该模式也有几个缺点。当使用全局变量或类的实例化非常耗资源但最终却没有用到他们的情况下，单例的影响可以忽略不计。